In [ ]:
# Let the games begin

import copy
from typing import Any

import torch
import wandb

from TicTacToe.Agent import RandomAgent
from TicTacToe.DeepQAgent import DeepQLearningAgent, DeepQPlayingAgent, ReplayBuffer
from TicTacToe.Evaluation import evaluate_performance

# os.environ["WANDB_MODE"] = "offline"
from TicTacToe.TicTacToe import TicTacToe

params: dict[str, Any] = {
    "nr_of_episodes": 500000,  # number of episodes for training
    "rows": 3,  # rows of the board, rows = cols
    "epsilon_start": 0.15,  # initial exploration rate
    "epsilon_min": 0.05,  # minimum exploration rate
    "learning_rate": 0.0001,  # learning rate
    "gamma": 0.95,  # discount factor
    "switching": True,  # switch between X and O
    # Parameters for DeepQAgent
    # "batch_size": 128,  # batch size for deep learning
    "batch_size": 256,  # batch size for deep learning
    "target_update_frequency": 20,  # target network update frequency
    "evaluation": True,  # save data for evaluation
    "double_q_learning": False,  # flag to switch on double Q-learnning
    "device": "cpu",  # device to use, 'cpu' or 'mps' or 'cuda'
    # "replay_buffer_length": 10000,  # replay buffer length
    "replay_buffer_length": 20000,  # replay buffer length
    "wandb": False,  # switch for logging with wand.ai
    "wandb_logging_frequency": 100,  # wandb logging frequency
    "load_network": False,  # file name of PyTorch network
    "shared_replay_buffer": False,  # shared replay buffer
}

# rows = 6
# win_length = 5
# nr_of_episodes = 750000
rows = 6
win_length = 5
nr_of_episodes = 25000
evaluation_frequency = 500
# shared_replay_buffer = ReplayBuffer(params["replay_buffer_length"], rows**2, device=params["device"])
# params["shared_replay_buffer"] = shared_replay_buffer
params["nr_of_episodes"] = nr_of_episodes
params["rows"] = rows

paramsX = copy.deepcopy(params)
paramsO = copy.deepcopy(params)
paramsX["player"] = "X"
paramsO["player"] = "O"
paramsX["wandb"] = True
paramsO["wandb"] = False

outcomes = {"X": 0, "O": 0, "D": 0}

learning_agent1 = DeepQLearningAgent(paramsX)
learning_agent2 = DeepQLearningAgent(paramsO)
# random_agent = RandomAgent(player="O", switching=True)

game = TicTacToe(learning_agent1, learning_agent2, display=None, rows=rows, cols=rows, win_length=win_length)

try:
    for episode in range(nr_of_episodes):
        outcome = game.play()
        if outcome is not None:
            outcomes[outcome] += 1

        if episode % evaluation_frequency == 0:
            evaluate_performance(learning_agent1, learning_agent2, rows=rows, win_length=win_length, wandb_logging=True)

    print("Outcomes during learning:")
    print(
        f"X wins: {outcomes['X']/nr_of_episodes}, O wins: {outcomes['O']/nr_of_episodes}, draws: {outcomes['D']/nr_of_episodes}"
    )

finally:
    torch.save(learning_agent1.q_network, f"../models/q_network_{rows}x{rows}x{win_length}_X.pth")
    torch.save(learning_agent2.q_network, f"../models/q_network_{rows}x{rows}x{win_length}_O.pth")

    wandb.finish()

In [ ]:
import numpy as np
import torch

q_network1 = learning_agent1.q_network

board1 = np.array(["X", "O", " ", " ", " ", " ", " ", " ", " "])
action1 = 2

board2 = np.array([" ", "O", "X", " ", " ", " ", " ", " ", " "])
action2 = 0

permutations, inverse_permutations = learning_agent1.generate_permutations([lambda x: x, lambda x: np.rot90(x, k=3)], 3)

state1 = torch.Tensor(learning_agent1.board_to_state(board1)[0])
state2 = torch.Tensor(learning_agent1.board_to_state(board2)[0])

board1 = np.array(["X", "O", "X", " ", " ", " ", " ", "O", " "])
action1 = 1
state1 = torch.Tensor(learning_agent1.board_to_state(board1)[0])

print(q_network1(state1))
print(q_network1(state1[permutations[1]]))
print(q_network1(state1)[permutations[1]])

print(torch.sum((q_network1(state1) - q_network1(state1[permutations[1]])) ** 2) / torch.sum((q_network1(state1)) ** 2))
print(
    torch.sum((q_network1(state1)[permutations[1]] - q_network1(state1[permutations[1]])) ** 2)
    / torch.sum((q_network1(state1)[permutations[1]]) ** 2)
)

In [ ]:
agent = DeepQLearningAgent(paramsX)
q_network1 = agent.q_network

board1 = np.array(["X", "O", "X", " ", " ", " ", " ", " ", " "])
action1 = 1
state1 = torch.Tensor(learning_agent1.board_to_state(board1)[0])

print(q_network1(state1))
print(q_network1(state1[permutations[1]]))
print(q_network1(state1)[permutations[1]])

print(torch.sum((q_network1(state1) - q_network1(state1[permutations[1]])) ** 2) / torch.sum((q_network1(state1)) ** 2))
print(
    torch.sum((q_network1(state1)[permutations[1]] - q_network1(state1[permutations[1]])) ** 2)
    / torch.sum((q_network1(state1)[permutations[1]]) ** 2)
)

In [ ]:
q_network1 = learning_agent1.q_network
playing_agent1 = DeepQPlayingAgent(q_network1, player="X", switching=False)
random_agent2 = RandomAgent(player="O", switching=False)

game = TicTacToe(playing_agent1, random_agent2, display=None, rows=rows, cols=rows, win_length=win_length)
nr_of_episodes = 1000
outcomes = {"X": 0, "O": 0, "D": 0}
for episode in range(nr_of_episodes):
    outcome = game.play()
    outcomes[outcome] += 1

print("Outcomes during playing:")
print(
    f"X wins: {outcomes['X']/nr_of_episodes}, O wins: {outcomes['O']/nr_of_episodes}, draws: {outcomes['D']/nr_of_episodes}"
)

q_network2 = learning_agent2.q_network
playing_agent2 = DeepQPlayingAgent(q_network2, player="O", switching=False)
random_agent1 = RandomAgent(player="X", switching=False)

game = TicTacToe(random_agent1, playing_agent2, display=None, rows=rows, cols=rows, win_length=win_length)
nr_of_episodes = 1000
outcomes = {"X": 0, "O": 0, "D": 0}
for episode in range(nr_of_episodes):
    outcome = game.play()
    outcomes[outcome] += 1

print("Outcomes during playing:")
print(
    f"X wins: {outcomes['X']/nr_of_episodes}, O wins: {outcomes['O']/nr_of_episodes}, draws: {outcomes['D']/nr_of_episodes}"
)

game = TicTacToe(playing_agent1, playing_agent2, display=None, rows=rows, cols=rows, win_length=win_length)
nr_of_episodes = 1000
outcomes = {"X": 0, "O": 0, "D": 0}
for episode in range(nr_of_episodes):
    outcome = game.play()
    outcomes[outcome] += 1

print("Outcomes during playing:")
print(
    f"X wins: {outcomes['X']/nr_of_episodes}, O wins: {outcomes['O']/nr_of_episodes}, draws: {outcomes['D']/nr_of_episodes}"
)

In [ ]:
# import torch

# torch.save(q_network1, 'models/q_network_4x4x4.pth')

In [ ]:
from Evaluation import plot_evaluation_data, plot_valid_actions

plot_evaluation_data(learning_agent1)
plot_evaluation_data(learning_agent2)

plot_valid_actions(learning_agent1)
plot_valid_actions(learning_agent2)